In [1]:
# ! pip3 install xgboost
import numpy as np
import pandas as pd
import xgboost as xgb
import gc
import numpy as np # for efficient numerical computations
import pandas as pd # for data management
import matplotlib.pyplot as plt #to customize visualizations
import seaborn as sns #for easy/common visualizations
# !pip install sklearn
#Importing sklearn libraries
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor

#Importing xgboost
from xgboost import XGBRegressor

import warnings
warnings.simplefilter("ignore")

#Importing other required libraries
import os
import re
import csv
import operator
import random
import datetime
from datetime import datetime
import warnings
color = sns.color_palette()
%matplotlib inline

random.seed(47)
# print'--------------------------------------------------'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/'My Drive'

/content/drive/My Drive


In [0]:
# !unzip zillow-prize-1.zip -d data

Archive:  zillow-prize-1.zip
replace data/zillow-prize-1/properties_2016.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/zillow-prize-1/properties_2016.csv  y

replace data/zillow-prize-1/properties_2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/zillow-prize-1/properties_2017.csv  
replace data/zillow-prize-1/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/zillow-prize-1/sample_submission.csv  
replace data/zillow-prize-1/train_2016_v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/zillow-prize-1/train_2016_v2.csv  
replace data/zillow-prize-1/train_2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/zillow-prize-1/train_2017.csv  
replace data/zillow-prize-1/zillow_data_dictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/zillow-prize-1/zillow_data_dictionary.xlsx  


In [0]:
train = pd.read_csv('data/zillow-prize-1/train_2016_v2.csv')
prop = pd.read_csv('data/zillow-prize-1/properties_2016.csv')
# prop = pd.read_csv('data/zillow-prize-1/properties_2016.csv')
sample = pd.read_csv('data/zillow-prize-1/sample_submission.csv')

In [5]:
train = pd.read_csv('data/zillow-prize-1/train_2016_v2.csv', parse_dates=["transactiondate"])
print ("Shape Of Train: ",train.shape)
train.head()

#Reading Properties Data
prop = pd.read_csv('data/zillow-prize-1/properties_2016.csv')
print ("Shape Of Train: ",prop.shape)
prop.head()

#Merging both files to create the train file
train = pd.merge(train, prop, on='parcelid', how='left')
print ("Shape Of Train: ",train.shape)
train.head()

#New Variable - Exploring number of transactions in each month
train['New_transaction_month'] = train['transactiondate'].dt.month

#Dropping Columns who's %age of NaN values is greater than 70%
for col in list(train.columns.values):
    nanValues = float(train[col].isnull().sum()) / len(train)
    if nanValues > 0.70:
        #print col
        train = train.drop([col], axis=1)
        
#Corelation Analysis, dropping highly correlated variables")
for col in ('calculatedbathnbr','fullbathcnt','finishedsquarefeet12','rawcensustractandblock','censustractandblock','landtaxvaluedollarcnt','taxvaluedollarcnt'):
    train = train.drop([col], axis=1)
    
#fill median for numeric null columns
for c in train.dtypes[train.dtypes == 'float64'].index.values:
    train[c].fillna(train[c].median(), inplace = True)
    
#Imputing Null values with -1 and encoding Objects
for c in train.columns:
    train[c]=train[c].fillna(-1)
    if train[c].dtype == object:
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values))
        train[c] = lbl.transform(list(train[c].values))
         
#Dropping out outliers
train=train[ train.logerror > -0.417 ]
train=train[ train.logerror < 0.418 ]

#Processing data
for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        train[c] = train[c].astype(np.float32)

#Splitting logerror as test data
test = train.logerror

#Dropping meta columns and casting to accepted dtypes
train = train.drop(['parcelid', 'logerror', 'transactiondate',
                'propertyzoningdesc', 'propertycountylandusecode'], axis=1)   
print('--------------------------------------------------')


Shape Of Train:  (90275, 3)
Shape Of Train:  (2985217, 58)
Shape Of Train:  (90275, 60)
--------------------------------------------------


In [0]:

#Adding new features

#proportion of living area
train['New_LivingAreaProp'] = train['calculatedfinishedsquarefeet']/train['lotsizesquarefeet']

#Number of properties in the zip
zip_count = train['regionidzip'].value_counts().to_dict()
train['New_zip_count'] = train['regionidzip'].map(zip_count)

#Number of properties in the city
city_count = train['regionidcity'].value_counts().to_dict()
train['New_city_count'] = train['regionidcity'].map(city_count)
# print'--------------------------------------------------'

In [7]:
#Printing out all the Variables
dtype_df = train.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print (dtype_df)

#Testing various Regression models
allModels = {} # Dictionary of models and their respective losses

print("Running XGB Regression")
model = XGBRegressor()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Linear Regression")
model = linear_model.LinearRegression()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Bayesian Ridge Regression")
model = linear_model.BayesianRidge()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Ridge Regression")
model = linear_model.Ridge()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Lasso Regression")
model = linear_model.Lasso()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Decision Trees")
model = tree.DecisionTreeRegressor()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Random Forest")
model = RandomForestRegressor()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running KNN")
model = KNeighborsRegressor()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Running Gradient Boosted Regressor")
model = GradientBoostingRegressor()
predicted = cross_val_score(model, train, test, scoring='neg_mean_absolute_error', cv=10)
allModels[model] = predicted.mean()

print("Returning the best model")
sortedModels = sorted(allModels.items(), key=operator.itemgetter(1), reverse=True)
for model in sortedModels:
    print ('Model:', model[0])
    print ('Loss:', model[1])
# print'--------------------------------------------------'

                           Count Column Type
0          airconditioningtypeid     float32
1                    bathroomcnt     float32
2                     bedroomcnt     float32
3          buildingqualitytypeid     float32
4   calculatedfinishedsquarefeet     float32
5                           fips     float32
6                   garagecarcnt     float32
7                garagetotalsqft     float32
8          heatingorsystemtypeid     float32
9                       latitude     float32
10                     longitude     float32
11             lotsizesquarefeet     float32
12         propertylandusetypeid     float32
13                  regionidcity     float32
14                regionidcounty     float32
15          regionidneighborhood     float32
16                   regionidzip     float32
17                       roomcnt     float32
18                       unitcnt     float32
19                     yearbuilt     float32
20    structuretaxvaluedollarcnt     float32
21        

In [8]:
print("Running Parameter optimization on the Best Model")

params = {  
            "n_estimators": [10,100,200,1000],
            "max_depth": [3,5,7],
            "min_child_weight": [1,3,5]
         }

xgb_model = XGBRegressor()  
xgb_cv    = GridSearchCV(xgb_model,params,verbose = 10,scoring='neg_mean_absolute_error')
xgb_cv.fit(train, test)  

print ('Best Score-', xgb_cv.best_score_)
print ('Best Parameters-', xgb_cv.best_params_)
print ('Best Estimator-', xgb_cv.best_estimator_)
# print'--------------------------------------------------'

Running Parameter optimization on the Best Model
Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] max_depth=3, min_child_weight=1, n_estimators=10 ................
[09:07:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=3, min_child_weight=1, n_estimators=10, score=-0.175, total=   0.9s
[CV] max_depth=3, min_child_weight=1, n_estimators=10 ................
[09:07:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  max_depth=3, min_child_weight=1, n_estimators=10, score=-0.182, total=   0.9s
[CV] max_depth=3, min_child_weight=1, n_estimators=10 ................
[09:07:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[CV]  max_depth=3, min_child_weight=1, n_estimators=100, score=-0.052, total=   7.4s
[CV] max_depth=3, min_child_weight=1, n_estimators=100 ...............
[09:08:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   34.2s remaining:    0.0s


[CV]  max_depth=3, min_child_weight=1, n_estimators=100, score=-0.052, total=   7.4s
[CV] max_depth=3, min_child_weight=1, n_estimators=200 ...............
[09:08:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  max_depth=3, min_child_weight=1, n_estimators=200, score=-0.055, total=  15.0s
[CV] max_depth=3, min_child_weight=1, n_estimators=200 ...............
[09:08:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  max_depth=3, min_child_weight=1, n_estimators=200, score=-0.054, total=  15.3s
[CV] max_depth=3, min_child_weight=1, n_estimators=200 ...............
[09:09:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  max_depth=3, min_child_weight=1, n_estimators=200, score=-0.052, total=  15.2s
[CV] max_depth=3, min_child_weight=1, n_estimators=200 ...........

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 123.8min finished


Best Score- -0.05313773974776268
Best Parameters- {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 100}
Best Estimator- XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=5, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [0]:
for c, dtype in zip(prop.columns, prop.dtypes):
	if dtype == np.float64:
		prop[c] = prop[c].astype(np.float32)

In [0]:
df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

(90275, 55) (90275,)


In [0]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

33

In [0]:
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

print('Building DMatrix...')

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

del x_train, x_valid; gc.collect()

print('Training ...')

Building DMatrix...
Training ...


In [0]:
params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

del d_train, d_valid

print('Building test set ...')

sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

[0]	train-mae:0.48808	valid-mae:0.481119
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.402199	valid-mae:0.395444
[20]	train-mae:0.332673	valid-mae:0.326099
[30]	train-mae:0.276517	valid-mae:0.270132
[40]	train-mae:0.231317	valid-mae:0.225214
[50]	train-mae:0.195064	valid-mae:0.189327
[60]	train-mae:0.166123	valid-mae:0.160725
[70]	train-mae:0.143115	valid-mae:0.13805
[80]	train-mae:0.124972	valid-mae:0.120241
[90]	train-mae:0.110785	valid-mae:0.106352
[100]	train-mae:0.099816	valid-mae:0.095695
[110]	train-mae:0.091452	valid-mae:0.087611
[120]	train-mae:0.085137	valid-mae:0.081602
[130]	train-mae:0.080447	valid-mae:0.077224
[140]	train-mae:0.077012	valid-mae:0.074089
[150]	train-mae:0.074499	valid-mae:0.07184
[160]	train-mae:0.072682	valid-mae:0.070257
[170]	train-mae:0.071361	valid-mae:0.069146
[180]	train-mae:0.070406	valid-mae:0.068383
[190]	train-mae:0.06971	valid-mae:

In [0]:
del prop; gc.collect()

x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

print('Predicting on test ...')

p_test = clf.predict(d_test)

del d_test; gc.collect()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Predicting on test ...


0

In [0]:
sub = pd.read_csv('data/zillow-prize-1/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

print('Writing csv ...')
sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f')

Writing csv ...


# New Section